In [1]:
import sys
import os

code_dir = os.environ.get('HALITE_PATH')

if not code_dir:
    code_dir = '/'.join(os.getcwd().split('/')[:-1] + ['rl_on_gcp'])
    
sys.path.append(code_dir)

In [2]:
from kaggle_environments import make
from kaggle_environments.envs.halite.helpers import Board, ShipAction, ShipyardAction, Observation
from halite_env import HaliteEnv
from ship_state_wrapper import ShipStateWrapper
from shipyard_state_wrapper import ShipYardStateWrapper
from agent import Agent

Loading environment football failed: No module named 'gfootball'


In [3]:
from halite_agent import HaliteAgent

In [4]:
environment = make('halite', {'episodeSteps': 250}, debug=True)

In [5]:
SHIP_MODEL_PATH = os.path.join(code_dir, 'my_model', 'ship_agent.h5')
SHIPYARD_MODEL_PATH = os.path.join(code_dir, 'my_model', 'shipyard_agent.h5')

In [6]:
ship_state_wrapper = ShipStateWrapper(
    radius=5,
    max_frames=2,
    map_size=int(environment.configuration['size'])
)
print(ship_state_wrapper.state_size)

shipyard_state_wrapper = ShipYardStateWrapper(
    radius=5,
    max_frames=1,
    map_size=int(environment.configuration['size'])
)
print(shipyard_state_wrapper.state_size)

732
365


In [7]:
ship_agent = Agent(
    alpha=0.001, gamma=0.9, n_actions=6,
    batch_size=32, 
    epsilon=0.01, 
    epsilon_end=0.01,
    input_dims=ship_state_wrapper.state_size,
    fname=SHIP_MODEL_PATH,
)

ship_agent.load_weights(SHIP_MODEL_PATH)

shipyard_agent = Agent(
    alpha=0.001, 
    gamma=0.9, 
    n_actions=2,
    batch_size=32, 
    epsilon=.01,
    input_dims=shipyard_state_wrapper.state_size,
    fname=SHIPYARD_MODEL_PATH,
)

shipyard_agent.load_weights(SHIPYARD_MODEL_PATH)

732
365


In [8]:
halite_env = HaliteEnv(
    opponents=1,
    ship_state_wrapper=ship_state_wrapper,
    shipyard_state_wrapper=shipyard_state_wrapper,
    radius=5,
    environment=None,
    trainer=None
)

In [9]:
halite_agent_opponent = HaliteAgent(
    configuration=environment.configuration,
    halite_env=halite_env,
    ship_agent=ship_agent,
    shipyard_agent=shipyard_agent,
    training=False,
    verbose=False,
    ship_frame_stack_len=2,
    ship_state_wrapper=ship_state_wrapper,
    shipyard_state_wrapper=shipyard_state_wrapper,
)

In [10]:
def call_opponent(observation, configuration):
    
    actions_for_step = {}
    
    raw_observation = observation
    step_observation = Observation(observation)
    
    halite_agent_opponent.env.ship_state_wrapper.set_map_size(configuration['size'])
        
    raw_observation, shipyard_simulated_step_memory = halite_agent_opponent.get_moves_for_all_shipyards(
        raw_observation=raw_observation,
        step_observation=step_observation,
        episode_number=1,
        step_number=1
    )

    raw_observation, ship_simulated_step_memory = halite_agent_opponent.get_moves_for_all_ships(
        raw_observation=raw_observation,
        step_observation=step_observation,
        episode_number=1,
        step_number=1
    )

    for id_, action in halite_agent_opponent.actions_for_step.items():
        actions_for_step[id_] = action

    return actions_for_step

In [11]:
from game_runner import GameRunner

In [12]:
episodes = 5
max_steps = 250

In [13]:
results = environment.run([call_opponent, "random"])

Ship 8-1 most likely collided, skipping this portion


environment.render(mode="ipython",width=800, height=600)